<a href="https://colab.research.google.com/github/flohmannjr/PJI410/blob/main/PJI410_projecoes_de_notificacoes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PJI410 - Projeções (time series) de notificações

# Importações

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import os

import tensorflow as tf

from tensorflow.data import AUTOTUNE, Dataset
from tensorflow.keras import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Add, Conv1D, Dense, Input, GRU, Lambda, Layer, LSTM, SimpleRNN, Subtract
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam

from absl import logging as absl_logging

# Configurações

In [ ]:
plt.rcParams['figure.figsize'] = [8, 5]
plt.rcParams['figure.dpi']     = 100

# plt.style.use('seaborn-darkgrid')
# plt.style.use('seaborn-v0_8-darkgrid')
sns.set_style("darkgrid")

SEMENTE = 2001088 + 2001247 + 2003061 + 2005493 + 2008193 + 2008620 + 2008880

JANELA_TAMANHO_INICIAL = 5
JANELA_TAMANHO_FINAL   = 7  # Não incluído

# JANELA_TAMANHO    = 6
HORIZONTE_TAMANHO = 1

CAMINHO_MARCOS = 'marcos'

# Define o nível de log Abseil para 'ERROR'.
# https://abseil.io/docs/python/guides/logging
absl_logging.set_verbosity(absl_logging.ERROR)

## Hiperparâmetros

In [ ]:
LOTE_TAMANHO = 32

CAMADAS = 4

CAMADA_UNIDADES      = 64
CAMADA_INICIALIZADOR = 'he_uniform'
CAMADA_ATIVACAO      = 'relu'

PROFUNDO_POTENCIA_INICIAL = 5
PROFUNDO_POTENCIA_FINAL   = 10  # Não incluída
PROFUNDO_REPETICAO        = 2
PROFUNDO_CONSOLIDACAO     = 32

SAIDA_UNIDADES = 1
SAIDA_ATIVACAO = 'linear'

PERDA       = 'mae'
OTIMIZADOR  = Adam
APRENDIZADO = 0.001
METRICAS    = ['mae', 'mse']

ITERACOES = 1000

## Funções

In [ ]:
!wget https://raw.githubusercontent.com/flohmannjr/PJI410/main/PJI410_funcoes_auxiliares.py

In [ ]:
from PJI410_funcoes_auxiliares import criar_janelas, criar_marco_modelo, grafico_series, metricas_modelo, separar_janelas_treino_teste, set_global_determinism

In [ ]:
def finalizar_modelo(modelo, modelo_nome, janelas_treino, horizontes_treino, janelas_teste, horizontes_teste):

    modelo.compile(loss=PERDA,
                   optimizer=OTIMIZADOR(learning_rate=APRENDIZADO),
                   metrics=METRICAS)

    modelo.fit(x=janelas_treino,
               y=horizontes_treino,
               epochs=ITERACOES,
               batch_size=LOTE_TAMANHO,
               validation_data=(janelas_teste, horizontes_teste),
               callbacks=[ReduceLROnPlateau(monitor='val_loss', patience=100, verbose=0),
                          criar_marco_modelo(modelo_nome, caminho=CAMINHO_MARCOS)],
               verbose=0)

    modelo    = load_model(os.path.join(CAMINHO_MARCOS, modelo_nome))
    previsoes = modelo.predict(janelas_teste, verbose=0)
    metricas  = metricas_modelo(horizontes_teste[:, -1], y_previsao=previsoes[:, -1])

    return modelo, metricas

# Dados

In [ ]:
dados = pd.read_csv('https://raw.githubusercontent.com/flohmannjr/PJI410/main/dados/PJI410_notificacoes_por_municipio.csv', parse_dates=['Segunda-feira'])
dados = dados[['Segunda-feira', 'Quantidade']].groupby(by=['Segunda-feira']).sum().sort_index().reset_index()

In [ ]:
q_treino = int(len(dados) * 0.8)

quantidades = dados['Quantidade'].to_numpy().astype('float32')

In [ ]:
X_train = dados[:q_treino]['Segunda-feira'].to_numpy()
y_train = dados[:q_treino]['Quantidade'].to_numpy()

X_test = dados[q_treino:]['Segunda-feira'].to_numpy()
y_test = dados[q_treino:]['Quantidade'].to_numpy()

In [ ]:
len(quantidades), len(X_train), len(y_train), len(X_test), len(y_test)

# Modelos

In [ ]:
modelos  = [None] * 50
metricas = [None] * 50

MODELO_NUMERO = 0

## Modelo Ingênuo

Naïve method: https://otexts.com/fpp3/simple-methods.html#na%C3%AFve-method

Para previsões ingênuas, simplesmente definimos todas as previsões como o valor da observação anterior.

In [ ]:
y_pred = y_test[:-1]

In [ ]:
# grafico_series(X_teste=X_test[1:], y_teste=y_test[1:],
#                X_previsao=X_test[1:], y_previsao=y_pred)

In [ ]:
metricas[MODELO_NUMERO] = metricas_modelo(y_test[1:], y_pred)

print(metricas[MODELO_NUMERO])

MODELO_NUMERO += 1

## Modelos Densos

In [ ]:
%%time

set_global_determinism(seed=SEMENTE)

for JANELA_TAMANHO in range(JANELA_TAMANHO_INICIAL, JANELA_TAMANHO_FINAL):

    janelas, horizontes = criar_janelas(quantidades, JANELA_TAMANHO, HORIZONTE_TAMANHO)

    janelas_treino, janelas_teste, horizontes_treino, horizontes_teste = separar_janelas_treino_teste(janelas, horizontes, tamanho_teste=0.2)

    modelo_nome = f'modelo_{MODELO_NUMERO}_denso_{JANELA_TAMANHO}j_{HORIZONTE_TAMANHO}h'

    modelos[MODELO_NUMERO] = Sequential(name=modelo_nome)

    for c in range(CAMADAS):
        modelos[MODELO_NUMERO].add(Dense(units=CAMADA_UNIDADES, kernel_initializer=CAMADA_INICIALIZADOR, activation=CAMADA_ATIVACAO, name=f'camada_oculta_{c}'))

    modelos[MODELO_NUMERO].add(Dense(units=HORIZONTE_TAMANHO, activation=SAIDA_ATIVACAO, name='camada_saida'))
    
    modelos[MODELO_NUMERO], metricas[MODELO_NUMERO] = finalizar_modelo(modelos[MODELO_NUMERO], modelo_nome, janelas_treino, horizontes_treino, janelas_teste, horizontes_teste)

    print(modelo_nome)
    print(metricas[MODELO_NUMERO])
    print()

    MODELO_NUMERO += 1

In [ ]:
# grafico_series(X_teste=X_test[-len(janelas_teste):], y_teste=horizontes_teste[:, -1],
#                X_previsao=X_test[-len(janelas_teste):], y_previsao=previsoes[:, -1])

## Modelos Densos Profundos

In [ ]:
%%time

set_global_determinism(seed=SEMENTE)

for JANELA_TAMANHO in range(JANELA_TAMANHO_INICIAL, JANELA_TAMANHO_FINAL):

    janelas, horizontes = criar_janelas(quantidades, JANELA_TAMANHO, HORIZONTE_TAMANHO)

    janelas_treino, janelas_teste, horizontes_treino, horizontes_teste = separar_janelas_treino_teste(janelas, horizontes, tamanho_teste=0.2)

    modelo_nome = f'modelo_{MODELO_NUMERO}_denso_profundo_{JANELA_TAMANHO}j_{HORIZONTE_TAMANHO}h'

    modelos[MODELO_NUMERO] = Sequential(name=modelo_nome)

    for potencia in range(PROFUNDO_POTENCIA_INICIAL, PROFUNDO_POTENCIA_FINAL):
        unidades = 2 ** potencia
        for r in range(PROFUNDO_REPETICAO):
            modelos[MODELO_NUMERO].add(Dense(units=unidades, kernel_initializer=CAMADA_INICIALIZADOR, activation=CAMADA_ATIVACAO, name=f'camada_oculta_{unidades}_{r}'))

    modelos[MODELO_NUMERO].add(Dense(units=PROFUNDO_CONSOLIDACAO, activation=CAMADA_ATIVACAO, name='camada_consolidacao'))
    modelos[MODELO_NUMERO].add(Dense(units=HORIZONTE_TAMANHO, activation=SAIDA_ATIVACAO, name='camada_saida'))
    
    modelos[MODELO_NUMERO], metricas[MODELO_NUMERO] = finalizar_modelo(modelos[MODELO_NUMERO], modelo_nome, janelas_treino, horizontes_treino, janelas_teste, horizontes_teste)

    print(modelo_nome)
    print(metricas[MODELO_NUMERO])
    print()

    MODELO_NUMERO += 1

In [ ]:
# grafico_series(X_teste=X_test[-len(janelas_teste):], y_teste=horizontes_teste[:, -1],
#                X_previsao=X_test[-len(janelas_teste):], y_previsao=previsoes[:, -1])

## Modelos Convulacionais

In [ ]:
%%time

set_global_determinism(seed=SEMENTE)

for JANELA_TAMANHO in range(JANELA_TAMANHO_INICIAL, JANELA_TAMANHO_FINAL):

    janelas, horizontes = criar_janelas(quantidades, JANELA_TAMANHO, HORIZONTE_TAMANHO)

    janelas_treino, janelas_teste, horizontes_treino, horizontes_teste = separar_janelas_treino_teste(janelas, horizontes, tamanho_teste=0.2)

    modelo_nome = f'modelo_{MODELO_NUMERO}_convulacional_{JANELA_TAMANHO}j_{HORIZONTE_TAMANHO}h'

    modelos[MODELO_NUMERO] = Sequential(name=modelo_nome)

    modelos[MODELO_NUMERO].add(Lambda(lambda x: tf.expand_dims(x, axis=1), name='camada_lambda'))  # Adiciona uma dimensão aos dados.

    modelos[MODELO_NUMERO].add(Conv1D(filters=CAMADA_UNIDADES,
                                      kernel_size=JANELA_TAMANHO,
                                      kernel_initializer=CAMADA_INICIALIZADOR,
                                      padding='causal',
                                      activation=CAMADA_ATIVACAO,
                                      name='camada_convulacional'))

    modelos[MODELO_NUMERO].add(Dense(units=HORIZONTE_TAMANHO, activation=SAIDA_ATIVACAO, name='camada_saida'))
    
    modelos[MODELO_NUMERO], metricas[MODELO_NUMERO] = finalizar_modelo(modelos[MODELO_NUMERO], modelo_nome, janelas_treino, horizontes_treino, janelas_teste, horizontes_teste)

    print(modelo_nome)
    print(metricas[MODELO_NUMERO])
    print()

    MODELO_NUMERO += 1

In [ ]:
# grafico_series(X_teste=X_test[-len(janelas_teste):], y_teste=tf.squeeze(horizontes_teste),
#                X_previsao=X_test[-len(janelas_teste):], y_previsao=tf.squeeze(previsoes))

## Modelos Recorrentes

In [ ]:
%%time

set_global_determinism(seed=SEMENTE)

recorrentes = {'rnn_simples': SimpleRNN, 'lstm': LSTM, 'gru': GRU}

for nome, funcao in recorrentes.items():

    for JANELA_TAMANHO in range(JANELA_TAMANHO_INICIAL, JANELA_TAMANHO_FINAL):

        janelas, horizontes = criar_janelas(quantidades, JANELA_TAMANHO, HORIZONTE_TAMANHO)

        janelas_treino, janelas_teste, horizontes_treino, horizontes_teste = separar_janelas_treino_teste(janelas, horizontes, tamanho_teste=0.2)

        modelo_nome = f'modelo_{MODELO_NUMERO}_{nome}_{JANELA_TAMANHO}j_{HORIZONTE_TAMANHO}h'

        modelos[MODELO_NUMERO] = Sequential(name=modelo_nome)

        modelos[MODELO_NUMERO].add(Lambda(lambda x: tf.expand_dims(x, axis=1), name='camada_lambda'))

        modelos[MODELO_NUMERO].add(funcao(units=CAMADA_UNIDADES,
                                          kernel_initializer=CAMADA_INICIALIZADOR,
                                          activation=CAMADA_ATIVACAO,  # Ativação 'relu' no lugar de 'tanh' por motivo de performance.
                                          name=f'camada_{nome}'))

        modelos[MODELO_NUMERO].add(Dense(units=CAMADA_UNIDADES, activation=CAMADA_ATIVACAO, name='camada_consolidacao'))
        modelos[MODELO_NUMERO].add(Dense(units=HORIZONTE_TAMANHO, activation=SAIDA_ATIVACAO, name='camada_saida'))
        
        modelos[MODELO_NUMERO], metricas[MODELO_NUMERO] = finalizar_modelo(modelos[MODELO_NUMERO], modelo_nome, janelas_treino, horizontes_treino, janelas_teste, horizontes_teste)

        print(modelo_nome)
        print(metricas[MODELO_NUMERO])
        print()

        MODELO_NUMERO += 1

In [ ]:
# grafico_series(X_teste=X_test[-len(janelas_teste):], y_teste=tf.squeeze(horizontes_teste),
#                X_previsao=X_test[-len(janelas_teste):], y_previsao=tf.squeeze(previsoes))

## Comparação dos modelos

In [ ]:
df_metricas = pd.DataFrame([m for m in metricas if m is not None])

with pd.option_context('display.max_columns', None):
    display(df_metricas.T)

In [ ]:
df_metricas['Mean Absolute Error'].sort_values()